## Ejercicio 1

In [130]:
#Importación de librerías requeridas para el funcionamiento del programa
import pandas as pd

In [6]:
#Variables Globales

# Ruta a archivo csv
ruta_csv = 'trips.csv'

FUNCION DE DESCARGA Y TRANSFORMACIÓN DE DATOS

In [7]:
#Función para carga de archivo csv y transformación a Dataframe

def descarga_data (ruta: str) -> pd.DataFrame:

    with open (ruta_csv) as csv:
        df = pd.read_csv(csv)
        return df

#Función para transformar data
def transformar_data (df: pd.DataFrame) -> pd.DataFrame:
    #Transformaciones para obtener latitud y longitud del punto/coordenada de origen
    # Se utiliza el método split para obtener coordenadas en columnas distintas y 
    # así poder agrupar por la parte entera o flotante de la coordenada, asumiendo que 
    # similares coordenas implica puntos de origen/destino similares
    origen = df.origin_coord.str.split(expand = True)
    origen = origen.rename(columns = {0:"point", 1:"lat_o", 2:"lon_o"})
    lat = origen.lat_o.str.split('(',expand=True)
    lat = lat.rename(columns = {0:"(", 1:"lat_origin"})
    lat = lat[["lat_origin"]]
    lon = origen.lon_o.str.split(')', expand=True)
    lon = lon.rename(columns = {0:"lon_origin", 1:")"})
    lon = lon[["lon_origin"]]
    origin_coord = pd.concat([lat, lon], axis=1)
    df_v1 = pd.concat([df, origin_coord], axis=1)
    #Transformaciones para obtener latitud y longitud del punto/coordenada de destino
    destino = df.destination_coord.str.split(expand = True)
    destino = destino.rename(columns = {0:"point", 1:"lat_d", 2:"lon_d"})
    lat_d = destino.lat_d.str.split('(',expand=True)
    lat_d = lat_d.rename(columns = {0:"(", 1:"lat_destination"})
    lat_d = lat_d[["lat_destination"]]
    lon_d = destino.lon_d.str.split(')', expand=True)
    lon_d = lon_d.rename(columns = {0:"lon_destination", 1:")"})
    lon_d = lon_d[["lon_destination"]]
    destination_coord = pd.concat([lat_d, lon_d], axis=1)
    df_v2 = pd.concat([df_v1, destination_coord], axis=1)
    
    fechahora = df.datetime
    fechahora = pd.to_datetime(fechahora, errors="coerce")
    fechahora = fechahora.rename(index="fechahora")
    df_v3 = pd.concat([df_v2, fechahora], axis=1)
    df_v3["hora"] = df_v3.fechahora.dt.hour
    #Me di cuenta que no podía simplemente hacer un split de la cadena, 
    # sino que debía manipular el string como flotante para obtener la parte entera 
    # y la parte decimal de cada coordenada (latitud y longitud) 
    #Se obtiene parte entera  y decimal de latitud de origen en columnas
    df_v3["lat_o_int"] = df_v3["lat_origin"].astype(float)
    df_v3["lat_o_int"] = df_v3["lat_o_int"].astype(int)
    df_v3["lat_o_float"] = df_v3["lat_origin"].astype(str).str.split('.').str[1]
    df_v3["lat_o_float"] = (df_v3["lat_o_float"]).str.slice(0, 1)

    #Se obtiene parte entera  y decimal de longitud de origen en columnas
    df_v3["lon_o_int"] = df_v3["lon_origin"].astype(float)
    df_v3["lon_o_int"] = df_v3["lon_o_int"].astype(int)
    df_v3["lon_o_float"] = df_v3["lon_origin"].astype(str).str.split('.').str[1]
    df_v3["lon_o_float"] = (df_v3["lon_o_float"]).str.slice(0, 1)

    #Se obtiene parte entera  y decimal de latitud de destino en columnas
    df_v3["lat_d_int"] = df_v3["lat_destination"].astype(float)
    df_v3["lat_d_int"] = df_v3["lat_d_int"].astype(int)
    df_v3["lat_d_float"] = df_v3["lat_destination"].astype(str).str.split('.').str[1]
    df_v3["lat_d_float"] = (df_v3["lat_d_float"]).str.slice(0, 1)

    #Se obtiene parte entera  y decimal de longitud de destino en columnas
    df_v3["lon_d_int"] = df_v3["lon_destination"].astype(float)
    df_v3["lon_d_int"] = df_v3["lon_d_int"].astype(int)
    df_v3["lon_d_float"] = df_v3["lon_destination"].astype(str).str.split('.').str[1]
    df_v3["lon_d_float"] = (df_v3["lon_d_float"]).str.slice(0, 1)
    #Finalmente para decidir la agrupación de datos hice pruebas de agrupación con la parte entera de latitud y longitud de las coordenadas 
    # y con la parte entera y flotante a la vez, pero al incluir la parte flotante, los casos similares eran sólo 2
    #Es de esperar que con una base más amplia tenga sentido considerar la parte flotante de las coordenadas en la agrupación...
    
    #OPCION 1 RETURN 
    #Se crea columna "cantidad_similar_en_region" para representar mediante un número entero la cantidad de veces que la fila agrupada por hora, longitud y latitud de origen y destino (parte entera), se repite en la base de datos
     #De esta forma se pueden filtrar las filas > 1, que serían aquellas que sí representan viajes similares
    df_v3["cantidad_similar_en_region"] = df_v3.groupby(['region','hora', 'lat_o_int', 'lon_o_int', 'lat_d_int','lon_d_int'])['region'].transform('count')
   
    #OPCION 2 RETURN 
    #para no aumentar el peso de la data podría limpiarse el dataframe y solo mantener las columnas transformadas:
    df_v4 = df_v3[['region', 'fechahora', 'hora', 'lat_origin', 'lat_o_int', 'lat_o_float', 'lon_origin', 'lon_o_int', 'lon_o_float', 'lat_destination', 'lat_d_int', 'lat_d_float', 'lon_destination', 'lon_d_int', 'lon_d_float', 'cantidad_similar_en_region']]
    
    #OPCION 3 
    #Otro enfoque sería simplemente a partir de las columnas creadas: hora, lat_o_int, lon_o_int, lat_d_int y lon_d_int, se aplique una función de agregación en el gestor de base de datos o datawarehouse donde se almacenará la totalidad de la data y no solo este archivo csv.
    
    return df_v3, df_v4

In [10]:
#COMENTARIOS EXTRAS A LA AGRUPACIÓN     

#Una forma de agrupar los datos era crear distintos dataframes según se cumplían las condiciones de coordenadas de origen
#y destino, y hora similares pero es impracticable ya que los valores de las variables constituyen un rango muy amplio

# df_0a4 = df_v3[(df_v3["hora"] < 5) & (df_v3["hora"] >= 0)]
# df_5a9 = df_v3[(df_v3["hora"] < 10) & (df_v3["hora"] >= 5)]
# df_10a14 = df_v3[(df_v3["hora"] < 15) & (df_v3["hora"] >= 10)]
# df_15a19 = df_v3[(df_v3["hora"] < 20) & (df_v3["hora"] >= 15)]
# df_20a24 = df_v3[(df_v3["hora"] < 25) & (df_v3["hora"] >= 20)]
# df_0a4["rango_horario"] = '0 a 4'
# df_5a9["rango_horario"] = '5 a 9'
# df_10a14["rango_horario"] = '10 a 14'
# df_15a19["rango_horario"] = '15 a 19'
# df_20a24["rango_horario"] = '20 a 24'

FUNCIÓN DE CARGA 

In [15]:
import pg8000
import sqlalchemy
from google.cloud.sql.connector import Connector

In [16]:
#Función auxiliar para establecer la conexión a la base de datos PostgreSql con conector Google CLoud SQL
def create_connection():
    POSTGRES_CONNECTION_NAME = "project_name:zone:dbsql_name"
    POSTGRES_USER = "postgres"
    POSTGRES_PASS = "postgres"
    POSTGRES_DB = "dbsql_name"

    connector = Connector()

    def init_connection_engine() -> sqlalchemy.engine.Engine:
        def getconn() -> pg8000.dbapi.Connection:
            conn: pg8000.dbapi.Connection = connector.connect(
                POSTGRES_CONNECTION_NAME,
                "pg8000",
                user=POSTGRES_USER,
                password=POSTGRES_PASS,
                db=POSTGRES_DB,
            )
            return conn

        engine = sqlalchemy.create_engine(
            "postgresql+pg8000://",
            creator=getconn,
        )
        engine.dialect.description_encoding = None
        return engine

    db = init_connection_engine()

    return db


In [17]:
def upload_data(df : pd.DataFrame):
    schema = "data_geografica"
    table = "viajes"

    connection = create_connection()

    connection.execute('''TRUNCATE TABLE {0}.{1}'''.format(schema, table))
    df.to_sql(table, connection, schema=schema, if_exists='append', method='multi', index=False)
 

Prueba funciones de descarga y transformación

In [8]:
#Se ejecuta función descarga_data()
df = descarga_data(ruta_csv)
#Se ejecuta función transformar_data()
dfinal_1, dfinal_2 = transformar_data(df) # Se obtienen dos df según las opciones que se presentaron anteriormente.

In [ ]:
#OPCIÓN 1 RETURN DE LA FUNCIÓN DESCARGA_DATA()

dfinal_1

In [ ]:
#OPCIÓN 2 RETURN DE LA FUNCIÓN DESCARGA_DATA()

dfinal_2

## Ejercicio 2

In [22]:
#Importación de librerías
from shapely.geometry import Polygon

In [ ]:
#A partir de la función descarga_data, se crea un nuevo df_coord con las columnas que se utilizarán para obtener el promedio por bounding box y región

In [149]:
#Se unifican coordenadas de longitud y latitud independiente de origen y destino para utlizar coordenadas en creación de bounding box
df_coord = dfinal_1[["region","lat_origin", "lon_origin"]]
df_coord_2= dfinal_1[["region","lat_destination", "lon_destination"]]
df_coord_2.rename(columns={"lat_destination":"lat_origin", "lon_destination": "lon_origin"},inplace=True)
df_coord = pd.concat([df_coord, df_coord_2],)
df_coord.rename(columns={"lat_origin":"latitud", "lon_origin": "longitud"},inplace=True)
df_coord = df_coord.astype({"latitud" :'float', "longitud":'float'})
df_coord.dtypes

C:\Users\Usuario\AppData\Local\Temp\ipykernel_15168\3069215760.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coord_2.rename(columns={"lat_destination":"lat_origin", "lon_destination": "lon_origin"},inplace=True)


In [151]:
#Función crea bounding box, la cual llamaremos en la funcion promedios_viajes() y le pasaremos los valores min y max de latitud y longitud por región
def bbox(long0, lat0, lat1, long1):
        return Polygon([[long0, lat0],
                        [long1,lat0],
                        [long1,lat1],
                        [long0, lat1]])

In [164]:
def promedios_viajes (df_coord):
    regiones = df_coord.region.unique()
    for r in regiones:
        contador = 0
        df_coor_region = df_coord[df_coord["region"]==r]
        max_lat = df_coord.latitud.max()
        min_lat = df_coord.latitud.min()
        max_lon = df_coord.longitud.min()
        min_lon = df_coord.longitud.max()
        bounding_box= bbox(max_lon, max_lon, min_lat, min_lon)
        total_filas = len(df_coor_region)
        for index, row in df_coor_region.iterrows():
            if min_lat < row["latitud"] < max_lat:
                contador +=1
            else: 
                pass
        promedio = contador/total_filas
        print(r + ' promedio = ' + str(promedio) + ' en bounding box ' + str(bounding_box))

Prueba función promedios_viajes()

In [165]:
promedios_viajes(df_coord)

Prague promedio = 0.9852941176470589 en bounding box POLYGON ((44.97612466562052 44.97612466562052, 53.65419620043016 44.97612466562052, 53.65419620043016 7.513135087952872, 44.97612466562052 7.513135087952872, 44.97612466562052 44.97612466562052))
Turin promedio = 0.9868421052631579 en bounding box POLYGON ((44.97612466562052 44.97612466562052, 53.65419620043016 44.97612466562052, 53.65419620043016 7.513135087952872, 44.97612466562052 7.513135087952872, 44.97612466562052 44.97612466562052))
Hamburg promedio = 1.0 en bounding box POLYGON ((44.97612466562052 44.97612466562052, 53.65419620043016 44.97612466562052, 53.65419620043016 7.513135087952872, 44.97612466562052 7.513135087952872, 44.97612466562052 44.97612466562052))
